# Laboratory of Computational Physics mod. B at Phisics of Data, University of Padova

__Authors:__

- Valeria Fioroni (University of Padova)
- Matteo Guida (University of Padova)
- Philipp Zehetner (University of Padova, Ludwig Maximilian University of Munich)

__Supervised by:__

- Professor Marco Zanetti (University of Padova, CERN)
- Professor Stefano Lacaprara (University of Padova, BELLE2)

## 1. The Signal

At the SuperKEKB paricle accelerator electrons and positrons collide with a center-of-momentum energy clos to the $ \Upsilon (4S) $ resonance. The cross section for the event $ e^+ e^- \rightarrow \Upsilon (4S) $ is 

$$ 
\sigma_{\Upsilon (4S)} = 1.1 \, nb
$$

The $ \Upsilon (4S) $ decays in one of two (almost) equally probable products: 
- $ \Upsilon (4S) \rightarrow B^+ B^- $
- $ \Upsilon (4S) \rightarrow B^0 \bar{B^0} $

For this analysis we are interested in the decay channel
$$
B^0 \rightarrow \eta' \left( \eta \left( \gamma \gamma \right) \pi^+ \pi^- \right) K^0_S \left( \pi^+ \pi^- \right)
$$

The branching fractions are as follows: 

- $ \mathcal{BF} \left( B^0 \rightarrow \eta' K^0 \right) = 6.6 * 10^{-5} $ 
- $ \mathcal{BF} \left( \eta' \rightarrow \eta \pi^+ \pi^- \right) = 0.43 $
- $ \mathcal{BF} \left( \eta \rightarrow \gamma \gamma \right) = 0.40 $
- $ P \left( K^0_S | K^0 \right) = 0.5 $
- $ \mathcal{BF} \left( K^0_S \rightarrow \pi^+ \pi^- \right) = 0.7 $

This leads to a complete branching ratio of 
$ \mathcal{BF} = 3.97 * 10^{-6} $ 

## 2. Our Data

Our Data consists of labelled data, that we used to train and test a _Deep Neural Network_ (DNN) and a _Boosted Decision Tree_ as well as unlabelled data (Data Challenge) in which we are searching for our signal. 

###  Labelled Data

The labelled data consists of various types of background, all corresponding to an integrated luminosity of 
$ \mathcal{L} = 800 \, fb^{-1} $
and signal corresponding to __NSIGNAL__ reconstructed signal events with an efficiency of 
$ \varepsilon = 0.291 $

- $ e^+ e^- \rightarrow u \bar{u} $
- $ e^+ e^- \rightarrow d \bar{d} $
- $ e^+ e^- \rightarrow s \bar{s} $
- $ e^+ e^- \rightarrow c \bar{c} $
- $ e^+ e^- \rightarrow \tau \bar{\tau} $
- $ e^+ e^- \rightarrow B^+ B^- $
- $ e^+ e^- \rightarrow B^0 \bar{B^0} $

We summarize the first five types of backgrounds as _Continuum Background_ and the last two types as _Peaking Background_ and thus build a multi classifier with three types of events. 

Using the complete branching fraction 
$ \mathcal{BF} $
and the cross section 
$ \sigma_{\Upsilon (4S)} $ 
we can calculate to expect to have $N$ events with 
\begin{equation}
N = \mathcal{L} \times \sigma_{\Upsilon(4S)} \times \mathcal{BF} 
= 800 \, fb^{-1} \times 1.1 \, nb \times 3.97 \cdot 10^{-6}
= 3494
\end{equation}
of which $ \varepsilon \times N = 1017 $ should be found by the reconstruction

###  Data Challenge

The Data Challenge is a simulation corresponding to a total luminosity of 
$ \mathcal{L} = 1 \, ab^{-1} $
We can calculate in the same as above, that we should expect a total number of $ N = 4367 $ signal events of which $1271$ are reconstructed. 

In [1]:
import os, sys, pprint
import math
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import joblib
import warnings
warnings.filterwarnings("ignore")
from root_pandas import read_root
from random import seed
from random import randint
n_seed=1234
seed(n_seed)

Local_Philipp = True
Local_Valeria = False

Welcome to JupyROOT 6.16/00


In [2]:
Training = ['B0_Pcms', 'B0_etap_Pcms', 'B0_etap_eta_Pcms', 'B0_etap_eta_gamma0_Pcms', 'B0_etap_eta_gamma1_Pcms', 'B0_etap_pi0_Pcms', 'B0_etap_pi1_Pcms', 'B0_K_S0_Pcms', 'B0_M', 'B0_ErrM', 'B0_SigM', 'B0_K_S0_M', 'B0_K_S0_ErrM', 'B0_K_S0_SigM', 'B0_etap_M', 'B0_etap_ErrM', 'B0_etap_SigM', 'B0_etap_eta_M', 'B0_etap_eta_ErrM', 'B0_etap_eta_SigM', 'B0_deltae', 'B0_mbc', 'B0_TrCSMVA', 'B0_X', 'B0_ErrX', 'B0_Y', 'B0_ErrY', 'B0_Z', 'B0_ErrZ', 'B0_Rho', 'B0_etap_X', 'B0_etap_ErrX', 'B0_etap_Y', 'B0_etap_ErrY', 'B0_etap_Z', 'B0_etap_ErrZ', 'B0_etap_Rho', 'B0_etap_eta_X', 'B0_etap_eta_ErrX', 'B0_etap_eta_Y', 'B0_etap_eta_ErrY', 'B0_etap_eta_Z', 'B0_etap_eta_ErrZ', 'B0_etap_eta_Rho', 'B0_etap_pi0_X', 'B0_etap_pi0_ErrX', 'B0_etap_pi0_Y', 'B0_etap_pi0_ErrY', 'B0_etap_pi0_Z', 'B0_etap_pi0_ErrZ', 'B0_etap_pi0_Rho', 'B0_etap_pi1_X', 'B0_etap_pi1_ErrX', 'B0_etap_pi1_Y', 'B0_etap_pi1_ErrY', 'B0_etap_pi1_Z', 'B0_etap_pi1_ErrZ', 'B0_etap_pi1_Rho', 'B0_K_S0_X', 'B0_K_S0_ErrX', 'B0_K_S0_Y', 'B0_K_S0_ErrY', 'B0_K_S0_Z', 'B0_K_S0_ErrZ', 'B0_K_S0_Rho', 'B0_cosAngleBetweenMomentumAndVertexVector', 'B0_distance', 'B0_significanceOfDistance', 'B0_dr', 'B0_etap_pi0_dr', 'B0_etap_pi1_dr', 'B0_K_S0_dr', 'B0_decayAngle__bo0__bc', 'B0_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo0__bc', 'B0_etap_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo2__bc', 'B0_VtxPvalue', 'B0_etap_VtxPvalue', 'B0_etap_eta_VtxPvalue', 'B0_etap_pi0_VtxPvalue', 'B0_etap_pi1_VtxPvalue', 'B0_K_S0_VtxPvalue']
Important = Training + ['B0_isSignal', 'evt_no']

if Local_Philipp : 
    path = '/home/philipp/Desktop/Project/DATA/'
    SFiles = glob.glob(os.path.join(path, 'Signal/*.root'))
    CFiles = glob.glob(os.path.join(path, 'Continuous/*.root'))
    PFiles = glob.glob(os.path.join(path, 'Peaking/*.root'))
    DCFiles = glob.glob(os.path.join(path, 'DC/*.root'))
    
if Local_Valeria : 
    path = '/home/utente/Scrivania/Progetto LCP-B/DataBelle2_all/'
    SFiles = glob.glob(os.path.join(path, 'Signal/*.root'))
    CFiles = glob.glob(os.path.join(path, 'Background/Continuum/*.root'))
    PFiles = glob.glob(os.path.join(path, 'Background/Peaking/*.root'))
    DCFiles = glob.glob(os.path.join(path, 'DataChallenge/*.root'))
    
Signal = pd.concat((read_root(f, 'B0', columns=Important) for f in SFiles))
Signal = Signal[Signal['B0_isSignal']==1].reset_index(drop=True)
Continuum_bkg = pd.concat((read_root(f, 'B0', columns=Important) for f in CFiles))
Peaking_bkg = pd.concat((read_root(f, 'B0', columns=Important) for f in PFiles))

#Signal: In case of more than 1 candidate per event select the first one occuring
Mask_duplicated = Signal.duplicated(subset='evt_no', keep='first')
Mask_duplicated=np.logical_not(Mask_duplicated)
Signal=Signal[Mask_duplicated]
Signal.reset_index(drop=True, inplace=True)

Signal=Signal[Training]
Continuum_bkg=Continuum_bkg[Training]
Peaking_bkg=Peaking_bkg[Training]

print("Continuum: {} events".format(Continuum_bkg.shape[0]))
print("Peaking: {} events".format(Peaking_bkg.shape[0]))
print("Signal: {} events \n".format(Signal.shape[0]))
print("Assuming, that originally 1.6M signal events existed, this corresponds to a reconstruction efficiencey of {}%".format(100*Signal.shape[0]/1600000))

Continuum: 246329 events
Peaking: 2167 events
Signal: 218596 events 

Assuming, that originally 1.6M signal events existed, this corresponds to a reconstruction efficiencey of 13.66225%


### Preprocessing our Data

- explain steps in preprocessing

In [3]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

n_classes = 3
Signal['Type'] = 2
Continuum_bkg['Type'] = 1
Peaking_bkg['Type'] = 0

# Splitting Data and Label into train, test and validation separatly for each class
X = Signal.drop('Type',axis=1)
Y = Signal['Type']
X_train_S, X_test_val_S, Y_train_S, Y_test_val_S = train_test_split(X, Y, train_size=0.5,random_state=randint(10**6,10**9))
X = X_test_val_S
Y = Y_test_val_S
X_validation_S, X_test_S, Y_validation_S, Y_test_S = train_test_split(X, Y, train_size=0.6,random_state=randint(10**6,10**9))
del(X_test_val_S)
del(Y_test_val_S)
X = Continuum_bkg.drop('Type',axis=1)
Y = Continuum_bkg['Type']
X_train_C, X_test_val_C, Y_train_C, Y_test_val_C = train_test_split(X, Y, train_size=0.5,random_state=randint(10**6,10**9))
X = X_test_val_C
Y = Y_test_val_C
X_validation_C, X_test_C, Y_validation_C, Y_test_C = train_test_split(X, Y, train_size=0.6,random_state=randint(10**6,10**9))
del(X_test_val_C)
del(Y_test_val_C)
X = Peaking_bkg.drop('Type',axis=1)
Y = Peaking_bkg['Type']
X_train_P, X_test_val_P, Y_train_P, Y_test_val_P = train_test_split(X, Y, train_size=0.5,random_state=randint(10**6,10**9))
X = X_test_val_P
Y = Y_test_val_P
X_validation_P, X_test_P, Y_validation_P, Y_test_P = train_test_split(X, Y, train_size=0.6,random_state=randint(10**6,10**9))
del(X_test_val_P)
del(Y_test_val_P)

# Concatenating the different classes, encoding the label as one-hot vectors and shuffling everything
X_Train=np.concatenate((X_train_S, X_train_C, X_train_P), axis=0)
X_Validation=np.concatenate((X_validation_S, X_validation_C, X_validation_P), axis=0)
X_Test=np.concatenate((X_test_S, X_test_C, X_test_P), axis=0)
Y_Train=np.concatenate((Y_train_S, Y_train_C, Y_train_P), axis=0)
Y_Validation=np.concatenate((Y_validation_S, Y_validation_C, Y_validation_P), axis=0)
Y_Test=np.concatenate((Y_test_S, Y_test_C, Y_test_P), axis=0)
Ỳ_Test_not_encoded=Y_Test #used for cut value analysis
Y_Train=to_categorical(Y_Train, num_classes=3)
Y_Validation=to_categorical(Y_Validation, num_classes=3)
Y_Test=to_categorical(Y_Test, num_classes=3)
permutation = np.random.permutation(X_Train.shape[0])
X_Train = X_Train[permutation]
Y_Train = Y_Train[permutation]
permutation = np.random.permutation(X_Validation.shape[0])
X_Validation = X_Validation[permutation]
Y_Validation = Y_Validation[permutation]
permutation = np.random.permutation(X_Test.shape[0])
X_Test = X_Test[permutation]
Y_Test = Y_Test[permutation]

Using TensorFlow backend.


Write something about the 5 sigma mass cut and why we aren't doing it
It's something we tried, so worth mentioning

Write something about scaling and why it is important

In [23]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
pcTrain = ss.fit_transform(X_Train)
pcTest = ss.transform(X_Test)
pcValidation = ss.transform(X_Validation)

In [5]:
X_Train.shape

(233545, 83)

Write something about PCA, and why we are not doing it

## 3. The Neural Network

explain what neural network we are using, how we have been training it

In [34]:
from keras.models import load_model

Model1 = load_model('Models/LastTry/Best_Model.h5')
Model2 = load_model('Models/LateNight/Best_Model.h5')

print(Model1.summary())
print(Model1.evaluate(pcTest, Y_Test))
print(2*'\n')
print(Model2.summary())
print(Model2.evaluate(pcTest, Y_Test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_148 (Dropout)        (None, 83)                0         
_________________________________________________________________
dense_662 (Dense)            (None, 50)                4200      
_________________________________________________________________
dense_663 (Dense)            (None, 100)               5100      
_________________________________________________________________
dense_664 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_665 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_666 (Dense)            (None, 3)                 153       
Total params: 24,603
Trainable params: 24,603
Non-trainable params: 0
_________________________________________________________________
None
9

Talk about the performance of our model 

- Model.evaluate with Test sample
- ROCK Curve
- Confusion Matrix

## 4. XGBOOST

- Same story here as in the last chapter
- Advantages/Disadvantages compared to the DNN

## 5. Deciding on the Cut Value of our Network

- all necessary steps and explanations for the cut value

## 6. The Data Challenge

Here we con showcase the perfect performance of our network

In [7]:
DC = pd.concat((read_root(f, 'B0', columns=Training) for f in DCFiles)).reset_index(drop=True)

In [40]:
pcDC = ss.transform(np.array(DC.values))
print(Model1.predict(pcDC))
print('\n', 50*'-', '\n')
print(Model2.predict(pcDC))

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]

 -------------------------------------------------- 

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [41]:
pcDC1 = ss.fit_transform(np.array(DC.values))
print(Model1.predict(pcDC1))
print('\n', 50*'-', '\n')
print(Model2.predict(pcDC1))

[[3.2186895e-03 9.8908037e-01 7.7009602e-03]
 [1.6471164e-02 9.8352885e-01 2.5534923e-09]
 [1.7694786e-02 9.8230529e-01 1.0872415e-09]
 ...
 [3.2141146e-03 1.7170277e-01 8.2508308e-01]
 [4.1421424e-03 2.5818574e-01 7.3767209e-01]
 [5.1302887e-03 8.9486539e-02 9.0538317e-01]]

 -------------------------------------------------- 

[[1.39860185e-02 9.48301852e-01 3.77121717e-02]
 [7.85904005e-03 9.92121935e-01 1.90740229e-05]
 [6.94253435e-03 9.93052006e-01 5.47205127e-06]
 ...
 [9.99862328e-03 5.39819971e-02 9.36019421e-01]
 [1.21847913e-02 7.03602433e-02 9.17454958e-01]
 [3.30546382e-03 1.39677212e-01 8.57017338e-01]]


In [47]:
Signal

,B0_Pcms,B0_etap_Pcms,B0_etap_eta_Pcms,B0_etap_eta_gamma0_Pcms,B0_etap_eta_gamma1_Pcms,B0_etap_pi0_Pcms,B0_etap_pi1_Pcms,B0_K_S0_Pcms,B0_M,B0_ErrM,...,B0_etap_decayAngle__bo0__bc,B0_etap_decayAngle__bo1__bc,B0_etap_decayAngle__bo2__bc,B0_VtxPvalue,B0_etap_VtxPvalue,B0_etap_eta_VtxPvalue,B0_etap_pi0_VtxPvalue,B0_etap_pi1_VtxPvalue,B0_K_S0_VtxPvalue,Type
0,0.337742,2.715602,1.822239,1.143091,0.745460,0.619497,0.319340,2.387300,5.307438,0.015551,...,0.568254,0.079900,-0.670724,6.053036e-04,4.801615e-02,5.644614e-01,5.343583e-01,0.361941,6.989494e-03,2
1,0.470735,2.385937,1.143156,0.478281,0.661026,0.587081,0.784182,2.633921,5.229204,0.032386,...,-0.527525,0.403934,0.673270,1.814637e-01,2.815941e-02,5.980884e-08,7.179064e-01,0.958767,9.333900e-01,2
2,0.374911,2.376649,1.349594,0.764921,0.606377,0.577851,0.491888,2.672966,5.267826,0.012465,...,-0.212969,0.754136,-0.192931,4.248845e-01,9.994511e-01,7.340705e-05,5.041033e-01,0.522255,2.304620e-01,2
3,0.353012,2.354433,1.863505,1.432046,0.510540,0.356780,0.190075,2.666672,5.242138,0.017393,...,0.806590,-0.218116,-0.850400,7.336494e-23,1.682415e-01,9.930433e-01,7.343598e-03,0.055590,9.996958e-01,2
4,0.370397,2.731697,1.909907,1.806603,0.192170,0.491492,0.396342,2.369550,5.302567,0.021979,...,0.708917,-0.190000,-0.458480,3.359036e-19,4.422435e-01,5.741555e-01,7.958414e-02,0.095357,1.686432e-12,2
5,0.315562,2.607498,1.671275,0.563308,1.228493,0.508353,0.459254,2.496872,5.314643,0.014977,...,0.221867,0.410579,-0.426582,7.519206e-01,2.232510e-01,1.874045e-01,3.291360e-02,0.815654,1.473663e-01,2
6,0.301456,2.668692,1.679291,1.622698,0.117909,0.417063,0.623882,2.397986,5.276021,0.027294,...,0.159948,-0.345820,0.042697,3.005651e-01,1.412901e-01,1.372220e-01,3.311472e-02,0.064277,3.209798e-01,2
7,0.340827,2.411188,1.012029,0.459017,0.665474,0.847501,0.565438,2.654195,5.283655,0.010152,...,-0.807682,0.801014,0.525782,1.876775e-01,6.124935e-01,1.923327e-01,2.180597e-01,0.122315,4.344848e-05,2
8,0.371696,2.587833,1.370511,0.878335,0.555065,0.441994,0.811336,2.492417,5.288472,0.011700,...,-0.339614,0.094515,0.208610,9.220702e-01,3.063509e-01,5.860644e-02,9.038852e-01,0.164531,2.030817e-01,2
9,0.335295,2.669212,1.905520,0.617216,1.316485,0.340512,0.461708,2.388990,5.265303,0.017716,...,0.795151,-0.634213,0.058788,7.358200e-01,7.423216e-01,6.221766e-02,4.463434e-01,0.431148,1.366209e-01,2


In [48]:
DC

,B0_K_S0_X,B0_K_S0_Rho,B0_K_S0_Pcms,B0_Pcms,B0_K_S0_SigM,B0_etap_pi1_Rho,B0_SigM,B0_etap_Z,B0_etap_pi1_X,B0_K_S0_VtxPvalue,...,B0_VtxPvalue,B0_etap_ErrY,B0_etap_eta_gamma1_Pcms,B0_mbc,B0_etap_pi0_Rho,B0_etap_pi0_ErrZ,B0_etap_X,B0_etap_pi0_Y,B0_ErrX,B0_Rho
0,1.719822,8.064223,2.227635,0.949089,1.211639,0.155719,2.453322,0.0,0.155438,6.857098e-01,...,5.392723e-04,0.0,0.053413,5.203940,0.036337,0.027404,0.0,-0.006757,0.002990,0.011985
1,5.555710,5.594461,2.539246,0.134671,0.011980,0.010196,2.489295,0.0,-0.000577,7.035756e-01,...,3.508110e-01,0.0,0.066420,5.288064,0.009962,0.008951,0.0,0.009732,0.000912,0.005585
2,5.555710,5.594461,2.539246,0.141258,0.011980,0.010196,1.764596,0.0,-0.000577,7.035756e-01,...,3.508110e-01,0.0,0.111560,5.287892,0.009962,0.008951,0.0,0.009732,0.000912,0.005585
3,5.555710,5.594461,2.539246,0.281482,0.011980,0.010196,2.738550,0.0,-0.000577,7.035756e-01,...,3.508110e-01,0.0,0.264485,5.282284,0.009962,0.008951,0.0,0.009732,0.000912,0.005585
4,5.555710,5.594461,2.539246,0.121087,0.011980,0.010196,-0.696838,0.0,-0.000577,7.035756e-01,...,3.508110e-01,0.0,0.086648,5.288393,0.009962,0.008951,0.0,0.009732,0.000912,0.005585
5,5.555710,5.594461,2.539246,0.211633,0.011980,0.010196,-6.094387,0.0,-0.000577,7.035756e-01,...,3.508110e-01,0.0,1.805176,5.285544,0.009962,0.008951,0.0,0.009732,0.000912,0.005585
6,5.555710,5.594461,2.539246,0.206744,0.011980,0.010196,-6.957547,0.0,-0.000577,7.035756e-01,...,3.508110e-01,0.0,1.805176,5.285737,0.009962,0.008951,0.0,0.009732,0.000912,0.005585
7,-0.207617,0.365994,2.468945,0.865624,0.863792,0.000028,0.411181,0.0,0.000025,4.064094e-01,...,4.569115e-15,0.0,0.693953,5.218472,0.025259,0.006311,0.0,-0.023719,0.000815,0.005143
8,-0.207617,0.365994,2.468945,0.842772,0.863792,0.000028,2.003570,0.0,0.000025,4.064094e-01,...,4.569115e-15,0.0,0.736592,5.222212,0.025259,0.006311,0.0,-0.023719,0.000815,0.005143
9,-0.018782,0.020061,2.813157,0.767780,-0.027094,0.003872,-1.197808,0.0,-0.001097,6.123806e-01,...,8.541521e-20,0.0,0.137029,5.233763,0.044498,0.008019,0.0,-0.020320,0.000700,0.000297


In [46]:
print(DC.columns)
print('\n', 50*'-', '\n')
print(Signal.columns)

Index(['B0_K_S0_X', 'B0_K_S0_Rho', 'B0_K_S0_Pcms', 'B0_Pcms', 'B0_K_S0_SigM',
       'B0_etap_pi1_Rho', 'B0_SigM', 'B0_etap_Z', 'B0_etap_pi1_X',
       'B0_K_S0_VtxPvalue', 'B0_Z', 'B0_etap_ErrZ', 'B0_K_S0_ErrX',
       'B0_K_S0_ErrY', 'B0_etap_pi0_dr', 'B0_etap_pi1_Z',
       'B0_decayAngle__bo0__bc', 'B0_K_S0_ErrZ', 'B0_etap_pi1_ErrY',
       'B0_etap_eta_Pcms', 'B0_distance', 'B0_etap_eta_ErrY', 'B0_etap_eta_Z',
       'B0_etap_pi1_Pcms', 'B0_etap_Rho', 'B0_etap_pi1_ErrZ',
       'B0_etap_pi1_ErrX', 'B0_etap_eta_Y', 'B0_M', 'B0_etap_M', 'B0_K_S0_dr',
       'B0_etap_pi0_ErrX', 'B0_X', 'B0_etap_Pcms', 'B0_etap_eta_gamma0_Pcms',
       'B0_etap_ErrX', 'B0_etap_eta_X', 'B0_etap_pi0_Pcms',
       'B0_decayAngle__bo1__bc', 'B0_etap_pi0_VtxPvalue', 'B0_ErrZ',
       'B0_K_S0_M', 'B0_etap_eta_Rho', 'B0_K_S0_Y', 'B0_TrCSMVA',
       'B0_etap_pi1_Y', 'B0_etap_pi1_dr', 'B0_etap_eta_ErrX', 'B0_Y',
       'B0_etap_decayAngle__bo0__bc', 'B0_etap_pi0_ErrY', 'B0_etap_eta_M',
       'B0_etap_VtxPva

## 7. Conclusion or additional comments (optional)

If you think somthing is missing feel free to add it. 
With 6 topics we can give 2 (non consecutive) topics for everybody. 
Having a little break between your two topics maybe helps a bit if something didn't go as expected. 